In [9]:
from deepface import DeepFace
import cv2
import numpy as np
import pandas as pd
import csv, os, re, time, logging, cv2, torch
from pathlib import Path
from importlib.metadata import version
from facenet_pytorch import InceptionResnetV1, MTCNN
from PIL import Image
from moviepy.editor import VideoFileClip


KeyboardInterrupt



In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
import glob
import random

# Constants
SEQUENCE_LENGTH = 30
STRIDE = 1
BATCH_SIZE = 32
SEQUENCES_PER_VIDEO = 50  # Number of random sequences to sample per video

# Helper function to create random sequences
def create_random_sequences(features, labels, sequence_length, num_sequences):
    x_sequences, y_sequences = [], []
    max_start = len(features) - sequence_length
    for _ in range(num_sequences):
        start_idx = random.randint(0, max_start)  # Random starting index
        x_sequences.append(features[start_idx:start_idx + sequence_length])
        y_sequences.append(labels[start_idx:start_idx + sequence_length])
    return np.array(x_sequences), np.array(y_sequences)

# Initialize lists to hold all sequences
all_x_sequences = []
all_y_sequences = []

# Define paths (modify these to match your file structure)
input_files = sorted(glob.glob('//153.19.52.107/emboa/IO3-sessions/NEW STRUCTURE/de-earlyfusionthesis/GUT/*_input.csv'))
label_files = sorted(glob.glob('//153.19.52.107/emboa/IO3-sessions/NEW STRUCTURE/de-earlyfusionthesis/GUT/*_BORIS.csv'))

# Process each pair of input and label files
for input_file, label_file in zip(input_files, label_files):
    # Load data
    input_df = pd.read_csv(input_file)
    label_df = pd.read_csv(label_file)
    
    # Prepare features and labels
    biosignals = input_df[['EDA', 'TEMP', 'HR']].values
    embeddings = input_df.iloc[:, 3:].values
    features = np.hstack((biosignals, embeddings))
    labels = label_df.values
    
    # Ensure alignment of frames
    if features.shape[0] != labels.shape[0]:
        print(f"Mismatch in frames: {input_file}, {label_file}")
        continue
    
    # Sample random sequences
    x_sequences, y_sequences = create_random_sequences(features, labels, SEQUENCE_LENGTH, SEQUENCES_PER_VIDEO)
    
    # Append to global lists
    all_x_sequences.append(x_sequences)
    all_y_sequences.append(y_sequences)

# Concatenate all sequences from all files
all_x_sequences = np.concatenate(all_x_sequences, axis=0)
all_y_sequences = np.concatenate(all_y_sequences, axis=0)

# Convert to TensorFlow dataset
dataset = tf.data.Dataset.from_tensor_slices((all_x_sequences, all_y_sequences))
dataset = dataset.shuffle(buffer_size=10000).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

# Verify the dataset
for x_batch, y_batch in dataset.take(1):
    print(f'Feature batch shape: {x_batch.shape}')
    print(f'Label batch shape: {y_batch.shape}')


Mismatch in frames: //153.19.52.107/emboa/IO3-sessions/NEW STRUCTURE/de-earlyfusionthesis/GUT\GUT_S02_C02_input.csv, //153.19.52.107/emboa/IO3-sessions/NEW STRUCTURE/de-earlyfusionthesis/GUT\GUT_S02_C02_BORIS.csv
Feature batch shape: (32, 30, 515)
Label batch shape: (32, 30, 6)


In [24]:
import os, re
from pathlib import Path
from moviepy.editor import VideoFileClip
def traverse():
    root_dir = '//153.19.52.107/emboa/IO3-sessions/NEW STRUCTURE'
    boris_ext = 'Analysis/BORIS/'
    type = ['Camera', 'Wristband']
    research_centers = ['GUT']
    #research_centers = ['GUT', 'ITU-YU', 'MAAP']
    #s_values = ["S01"]
    s_values = ["S01", "S02", "S03", "S04", "S05", "S06", "S07", "S08", "S09", "S10", "S11" ]
    #c_values = ["C01"]
    c_values = ["C01", "C02", "C03", "C04", "C05", "C06", "C07", "C08", "C09", "C10", "C11", "C12", "C13" ]
    for research_center in research_centers:
        for session in s_values:
            # Create the session path for biosignal and camera + BORIS
            camera_path = Path(root_dir).joinpath(type[0], research_center, session)
            signals_path = Path(root_dir).joinpath(type[1], research_center, session)
            if camera_path.is_dir() and signals_path.is_dir():
                # Create the meeting path for biosignal and camera + BORIS
                for camera in c_values:
                    exact_camera_path = Path(camera_path).joinpath(camera)
                    exact_signals_path = Path(signals_path).joinpath(camera)
                    if exact_camera_path.is_dir() and exact_signals_path.is_dir():
                        boris_path = Path(exact_camera_path).joinpath(boris_ext)
                        if boris_path.is_dir():
                            # Quick fix of two MAAP sessions being divided TODO: Fix this
                            if not(research_center == 'MAAP' and ((session == 'S01' and camera == 'C05') or (session == 'S03' and camera == 'C05'))):
                                mp4_video_path = glob.glob(os.path.join(exact_camera_path, "*.mp4"))[0]
                                with VideoFileClip(mp4_video_path) as video:
                                    mp4_duration = video.duration
                                mts_video_path = glob.glob(os.path.join(exact_camera_path, "*.MTS"))[0]
                                with VideoFileClip(mts_video_path) as video:
                                    mts_duration = video.duration
                                if mp4_duration > mts_duration:
                                    print(mp4_video_path)
                                else:
                                    print(mts_video_path)
                                    
traverse()


\\153.19.52.107\emboa\IO3-sessions\NEW STRUCTURE\Camera\GUT\S01\C01\Untitled 139.mp4
\\153.19.52.107\emboa\IO3-sessions\NEW STRUCTURE\Camera\GUT\S01\C02\00220.MTS
\\153.19.52.107\emboa\IO3-sessions\NEW STRUCTURE\Camera\GUT\S01\C03\00221.MTS
\\153.19.52.107\emboa\IO3-sessions\NEW STRUCTURE\Camera\GUT\S02\C01\Untitled 145.mp4
\\153.19.52.107\emboa\IO3-sessions\NEW STRUCTURE\Camera\GUT\S02\C02\00223.MTS
\\153.19.52.107\emboa\IO3-sessions\NEW STRUCTURE\Camera\GUT\S02\C03\Untitled 143.mp4
\\153.19.52.107\emboa\IO3-sessions\NEW STRUCTURE\Camera\GUT\S03\C01\00225.MTS
\\153.19.52.107\emboa\IO3-sessions\NEW STRUCTURE\Camera\GUT\S03\C02\00226.MTS
